In [2]:
import tensorflow as tf
import os

In [3]:
# Dataset directory and classes
dataset_dir = 'private/tomatect/'
classes = ['bacterial_spot', 'early_blight', 'healthy', 'leaf_mold', 'spider_mites']

In [12]:
# # Define image preprocessing function for augmentation
# def preprocess_image(image):
#     image = tf.image.resize(image, (256, 256))
#     image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_flip_up_down(image)
#     image = tf.keras.layers.experimental.preprocessing.RandomRotation(image, 20)
#     image = tf.image.random_brightness(image, max_delta=0.2)
#     image = tf.clip_by_value(image, 0, 1)
#     return image

data_scaling = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(256, 256),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
   tf.keras.layers.experimental.preprocessing.RandomRotation(0.9),
   tf.keras.layers.experimental.preprocessing.RandomContrast(0.5),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
  tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
  tf.keras.layers.experimental.preprocessing.RandomWidth(0.2)    
])

In [8]:
# Create dataset from directory and apply augmentation
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(256, 256),
    batch_size=32,
)

Found 250 files belonging to 5 classes.
Using 200 files for training.


In [13]:
# Apply preprocessing and caching to the dataset
dataset = dataset.map(
    lambda x, y: (data_augmentation(x), y),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
)
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
# Split dataset into train and validation
train_dataset = dataset.take(round(0.8 * len(dataset)))
val_dataset = dataset.skip(round(0.8 * len(dataset)))

In [19]:
# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

In [20]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10


2023-06-07 00:26:39.421076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-07 00:26:39.753590: I tensorflow/core/common_runtime/executor.cc:1210] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: In[0] mismatch In[1] shape: 96768 vs. 115200: [32,96768] [115200,256] 0 0
	 [[{{node sequential_3/dense_2/MatMul}}]]
2023-06-07 00:26:39.753668: I tensorflow/core/common_runtime/executor.cc:1210] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: In[0] mismatch In[1] shape: 96768 vs. 115200: [32,96768] [115200,256] 0 0
	 [[{{node sequential_3/dense_2/MatMul}}]]
	 [[sequential_3/dense_4/Softmax/_22]]
2023-06-07 00:26:39.753786: I tensorflow/core/framework/local_rendezvous.cc:

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_3/dense_2/MatMul' defined at (most recent call last):
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-pa

In [24]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

# Dataset directory and classes
dataset_dir = 'private/tomatect/'
classes = ['bacterial_spot', 'early_blight', 'healthy', 'leaf_mold', 'spider_mites']

# Load and preprocess the images
images = []
labels = []

for class_idx, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    for img_name in os.listdir(class_dir):
        if img_name.endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(class_dir, img_name)
            img = Image.open(img_path).convert("RGB")
            img = img.resize((256, 256))
            img_array = np.array(img) / 255.0  # Normalize the image
            images.append(img_array)
            labels.append(class_idx)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split the dataset into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create tf.data.Dataset from the numpy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

# Apply augmentation to the train dataset
augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
])

train_dataset = train_dataset.map(lambda x, y: (augmentation(x, training=True), y))

# Configure the datasets for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(len(X_train)).batch(32).prefetch(AUTOTUNE)
val_dataset = val_dataset.cache().batch(32).prefetch(AUTOTUNE)

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10


2023-06-07 00:30:17.138865: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [200]
	 [[{{node Placeholder/_1}}]]
2023-06-07 00:30:17.139586: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype resource
	 [[{{node Placeholder/_4}}]]
2023-06-07 00:30:17.473832: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-07 00:30:17.559640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - ETA: 0s - loss: 3.3211 - accuracy: 0.2400

2023-06-07 00:30:24.592070: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [50]
	 [[{{node Placeholder/_1}}]]
2023-06-07 00:30:24.841437: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 8s 714ms/step - loss: 3.3211 - accuracy: 0.2400 - val_loss: 1.6021 - val_accuracy: 0.2000
Epoch 2/10
7/7 [==============================] - 4s 527ms/step - loss: 1.6094 - accuracy: 0.2150 - val_loss: 1.5934 - val_accuracy: 0.1400
Epoch 3/10
7/7 [==============================] - 4s 516ms/step - loss: 1.5378 - accuracy: 0.2450 - val_loss: 1.4685 - val_accuracy: 0.3600
Epoch 4/10
7/7 [==============================] - 4s 539ms/step - loss: 1.4563 - accuracy: 0.3350 - val_loss: 1.3243 - val_accuracy: 0.3800
Epoch 5/10
7/7 [==============================] - 4s 546ms/step - loss: 1.3154 - accuracy: 0.4000 - val_loss: 1.6474 - val_accuracy: 0.2600
Epoch 6/10
7/7 [==============================] - 4s 564ms/step - loss: 1.3520 - accuracy: 0.3950 - val_loss: 1.5289 - val_accuracy: 0.3600
Epoch 7/10
7/7 [==============================] - 3s 487ms/step - loss: 1.2488 - accuracy: 0.5250 - val_loss: 1.2516 - val_accuracy: 0.4600
Epoch 8/10
7/7 [===============